In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8ed4aa39-9555-4841-82e8-7fab88343a37;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 452ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [2]:
df = spark.read.option('delimiter', ';').option('header', 'true').option("encoding","ISO-8859-1").csv('s3a://catarina-3sprint-sptech-bucket-tratados/twitter.csv')

22/06/15 21:31:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show(10)

+---+--------------------+
| id|             Twitter|
+---+--------------------+
|  1|     agua ruim de sp|
|  2|Casal alerta que ...|
|  3|Casal alerta que ...|
|  4|A qualidade da ág...|
|  5|"Faz 4 meses que ...|
|  6|@fredbernardes @Z...|
|  7|?? Caixa de Som J...|
|  8|Qualidade da água...|
|  9|Qualidade da água...|
| 10|Nota de esclareci...|
+---+--------------------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

_df = df.select(
    col('id').alias('ID'),
    col('Twitter').alias('TWITTER'),
       
)

 
# _df.write \
#    .mode('overwrite') \
#    .csv('s3a://marise-3cco/vendas_anuais')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://catarina-sprint3.database.windows.net;databaseName=catarina-sprint3;') \
 .option('dbtable', 'TWITTER') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()